In [ ]:
from test_fair_clustering import main
import argparse
import os
import csv
import numpy as np
import json

In [ ]:
OUTPUT_FOLDER = "outputs"
CSV_NAME = "results.csv"
CSV_NAME_LIPSCHITZ = "results_Lipschitz.csv"

lambdas_by_setting = {
    "Synthetic": {
        "kmedian": 600, # Theirs: 10 got consistently bad results
        "kmeans": 100, # Theirs: 10 got optimal results only on some seeds
        # "ncut": 10,
    }, 
    "Synthetic-unequal": {
        "kmedian": 10,
        "kmeans": 10, # 20,22,24 are outliers
        # "ncut": 10,
    }, 
    "Adult": {
        "kmedian": 9000,
        "kmeans": 9000,
        # "ncut": 1.5,
        # "ncut": 10, # Theirs with Lipschitz 1.0
    }, 
    "Bank": {
        "kmedian": 9000,
        "kmeans": 6000,
        # "ncut": 1,
        # "ncut": 40, # Theirs with Lipschitz 1.0
    }, 
    "CensusII": {
        # "kmedian": 500000,
        # "kmeans": 500000,
        # "ncut": 2.5,
        "ncut": 100, # Theirs with Lipschitz 1.0
    }
}

n_runs = { # Number of runs to do per dataset
    "Synthetic": 30, 
    "Synthetic-unequal": 30, 
    "Adult": 10,
    "Bank": 10,
    "CensusII": 5,
}
n_runs_Lipschitz = 10
Lipschitz_constants = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 2.0]

In [ ]:
def get_args(seed=1, dataset="Synthetic-unequal", cluster_option="ncut", lmbda=10, lmbda_tune=False, Lipschitz=None, plot_bound_update=False):
    """
    Return Namespace instance with passed and default arguments for main().

    plot_bound_update:
        if True, lets main() return a list of energies of a single bound update. Meant for checking convergence.
    """
    args = argparse.Namespace()
    
    args.plot_option_clusters_vs_lambda = False     # Only available for the synthetic datasets
    args.plot_option_fairness_vs_clusterE = True    # Only works when lmbda_tune == True
    args.plot_option_balance_vs_clusterE = False    # Only works when lmbda_tune == True
    args.plot_option_convergence = False
    args.plot_bound_update = plot_bound_update      # Return a list of the energies of a single bound update
    
    args.reprod = False
    args.kernel_type = None
    args.kernel_args = [1, 2]

    args.seed = seed
    args.dataset = dataset
    args.cluster_option = cluster_option
    args.lmbda = lmbda
    args.lmbda_tune = lmbda_tune
    
    if Lipschitz:
        args.L = Lipschitz
    elif cluster_option == "ncut" and "Synthetic" not in dataset:
        args.L = 1.0
    else:
        args.L = 2.0

    working_dir = os.getcwd()
    args.data_dir = os.path.join(working_dir, "data")
    args.output_path = os.path.join(working_dir, OUTPUT_FOLDER)
    return args

def make_csv(dir_path, csv_path, fieldnames):
    """
    Make path and csv file with header if it doesn't exist yet. Otherwise do nothing.

    dir_path:
        full path to directory
    csv_path:
        full path to directory including the csv file
    fieldnames:
        fieldnames for the csv
    """
    os.makedirs(dir_path, exist_ok=True)
    if os.path.isfile(csv_path):
        with open(csv_path, "r") as f:
            reader = csv.reader(f)
            if sum(1 for row in reader) > 0:
                return

    with open(csv_path, "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames)
        writer.writeheader()

def run_main(args, csv_name=CSV_NAME):
    """
    Run main and append results with settings to csv file.
    File is automatically made if it doesn't exist yet.

    csv_name:
        just the filename of the csv, eg "results.csv"
    """
    results = main(args, logging=False, seedable=True)

    save_dict = {
        "dataset": args.dataset,
        "N": results['N'],                                      # Dataset size
        "J": results['J'],                                      # Number of demographic groups (defined in dataset_load.py)
        "lmbda": args.lmbda,
        "Objective": results["clustering energy (Objective)"],  # Discrete clustering energy
        "fairness error": results["fairness error"],
        "balance": results["balance"],
        "cluster_option": args.cluster_option,
        "time": results["time"],                                # Time taken to finish this run
        "seed": args.seed,
        "lmbda_tune": args.lmbda_tune,                          
        "K": results['K'],                                      # Number of clusters (defined in dataset_load.py)
        "L": args.L,                                            # Lipschitz constant
    }

    csv_path = os.path.join(args.output_path, csv_name)
    fieldnames = save_dict.keys()
    make_csv(args.output_path, csv_path, fieldnames)
    with open(csv_path, "a", newline='') as f:
        writer = csv.DictWriter(f, fieldnames)
        writer.writerow(save_dict)
    

In [ ]:
def compare_entry(args, entry, keys):
    """
    Compare args to an entry from the csv file and return True if they have the given keys in common:

    args:
        arguments that will be passed to main()
    entry:
        entry in the csv file made by run_main()
    """
    for key in keys:
        if str(getattr(args, key)) != entry[key]:
            return False
    return True

def find_same_options(csv_name, args, keys=["dataset", "lmbda", "cluster_option", "lmbda_tune", "L"]):
    """
    Return entries from the csv file that have the same settings as the passed args, using compare_entry().

    csv_name:
        filename of the csv, eg "results.csv"
    args:
        arguments that will be passed to main()
    """
    entries = []
    csv_path = os.path.join(args.output_path, csv_name)
    with open(csv_path, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if compare_entry(args, row, keys):
                entries.append(row)
    return entries

def filter_outliers(entries, keys):
    pass

In [ ]:
"""Execute one run"""
seed = 1
dataset = "Synthetic-unequal"
cluster_option = "kmedian"
lmbda = lambdas_by_setting[dataset][cluster_option] # Use standard lambda
# lmbda = 1
lmbda_tune = False
L = 2.0 # Lipschitz constant
run_main(get_args(seed, dataset, cluster_option, lmbda, lmbda_tune, L))

In [ ]:
"""Run all settings with different seeds"""
for dataset in lambdas_by_setting:
    for cluster_option in lambdas_by_setting[dataset]:
        lmbda = lambdas_by_setting[dataset][cluster_option]

        args = get_args(dataset=dataset, cluster_option=cluster_option, lmbda=lmbda)
        existing_entries = find_same_options(CSV_NAME, args)

        if len(existing_entries) >= n_runs[dataset]:
            print("enough results for these settings")
            continue
        n_todo = n_runs[dataset] - len(existing_entries)

        seeds = [int(entry["seed"]) for entry in existing_entries]
        seeds.append(0) # Make sure seeds is not empty
        next_seed = max(seeds) + 1

        for new_seed in range(next_seed, next_seed + n_todo):
            args.seed = new_seed
            print()
            run_main(args, CSV_NAME)

In [ ]:
"""Fetch results from the csv"""
for dataset in lambdas_by_setting:
    print(f"\n\n{dataset}")
    for cluster_option in lambdas_by_setting[dataset]:
        print("\n  "+cluster_option.upper())
        lmbda = lambdas_by_setting[dataset][cluster_option]

        args = get_args(dataset=dataset, cluster_option=cluster_option, lmbda=lmbda)
        existing_entries = find_same_options(CSV_NAME, args)
        
        if len(existing_entries) < 1:
            print("no data yet")
            continue

        entry = existing_entries[0]
        print(f"  lmbda={lmbda}, Lipschitz={entry['L']}")
        
        keys = ["Objective", "fairness error", "balance", "time"]
        for key in keys:
            data = [float(entry[key]) for entry in existing_entries]
            mean = np.mean(data)
            std = np.std(data)

            print(f"    {key}{' '*(20-len(key))} M = {mean:.2f}     SD = {std:.2f}")

In [ ]:
def run_Lipschitz_test(args, csv_name=CSV_NAME_LIPSCHITZ):
    """
    Run bound update and append results with settings to csv file.
    File is automatically made if it doesn't exist yet.
    Logs of energy values by iteration are saved to .txt files as json, the path to which is saved in a csv field.

    csv_name:
        just the filename of the csv, eg "results_Lipschitz.csv"
    """
    args.plot_bound_update = True
    bound_energy_list, elapsed = main(args, logging=False, seedable=True)

    list_dir = os.path.join(args.dataset, "Lipschitz_energy_list_logs")
    os.makedirs(os.path.join(args.output_path, list_dir), exist_ok=True)
    list_filepath = os.path.join(list_dir, f"{args.cluster_option}__L={args.L}__lmb={args.lmbda}__seed={args.seed}.txt")
    list_filepath_full = os.path.join(args.output_path, list_filepath)
    with open(list_filepath_full, "w") as f:
        f.write(json.dumps(bound_energy_list))

    save_dict = {
        "dataset": args.dataset,
        "lmbda": args.lmbda,
        "cluster_option": args.cluster_option,
        "L": args.L,                            # Lipschitz constant
        "convergence_iter": len(bound_energy_list),
        "optimum": min(bound_energy_list),
        "time": elapsed,                        # Time taken to finish this run
        "seed": args.seed,
        "energy_list_file": list_filepath,        
    }

    csv_path = os.path.join(args.output_path, csv_name)
    fieldnames = save_dict.keys()
    make_csv(args.output_path, csv_path, fieldnames)
    with open(csv_path, "a", newline='') as f:
        writer = csv.DictWriter(f, fieldnames)
        writer.writerow(save_dict)

In [ ]:
# run_Lipschitz_test(get_args(Lipschitz=0.0001, plot_bound_update=True))

In [ ]:
"""Run Lipschitz test for multiple settings"""

for dataset in ["Synthetic", "Synthetic-unequal", "Adult", "Bank"]:
    for cluster_option in ["kmedian", "kmeans", "ncut"]:
        for L in Lipschitz_constants:

            args = get_args(dataset=dataset, cluster_option=cluster_option, lmbda=1, Lipschitz=L, plot_bound_update=True)
            existing_entries = find_same_options(CSV_NAME_LIPSCHITZ, args, keys=["dataset", "cluster_option", "L"])

            if len(existing_entries) >= n_runs_Lipschitz:
                print("enough results for these settings")
                continue
            n_todo = n_runs_Lipschitz - len(existing_entries)

            seeds = [int(entry["seed"]) for entry in existing_entries]
            seeds.append(0) # Make sure seeds is not empty
            next_seed = max(seeds) + 1

            for new_seed in range(next_seed, next_seed + n_todo):
                args.seed = new_seed
                print()
                run_Lipschitz_test(args, CSV_NAME_LIPSCHITZ)

In [ ]:
from matplotlib import pyplot as plt

def plot_Lipschitz_convergence(save_path, energy_list_by_L, yscale_log=False):
    """Plot clustering objective by iteration for different Lipschitz constants"""
    for L, lis in energy_list_by_L.items():
        plt.plot(range(len(lis["mean"])), lis["mean"], label=f"L = {L}")
        # plt.fill_between(range(len(lis["std"])), lis["mean"]+lis["std"], lis["mean"]-lis["std"], alpha=.5)
    plt.legend()
    plt.ylabel("fair objective")
    plt.xlabel("iterations")
    suffix = ""
    if yscale_log:
        plt.ylim(min(energy_list_by_L[2.0]["mean"]), np.max([max(lis["mean"][1:]) for lis in energy_list_by_L.values()]))
        plt.yscale('log')
        suffix = "_y-log"
    else:
        plt.ylim(min(energy_list_by_L[2.0]["mean"]), max(energy_list_by_L[0.01]["mean"][1:]))
    plt.xscale('log')
    plt.savefig(save_path.format(suffix=suffix))
    plt.close('all')

In [ ]:
"""Fetch Lipschitz results from the csv and make plots"""
for dataset in lambdas_by_setting:
    print("\n\n"+dataset)
    for cluster_option in lambdas_by_setting[dataset]:
        print("\n  "+cluster_option.upper())
        energy_list_by_L = {}
        complete = True
        for L in Lipschitz_constants:
            args = get_args(dataset=dataset, cluster_option=cluster_option, Lipschitz=L)
            existing_entries = find_same_options(CSV_NAME_LIPSCHITZ, args, keys=["dataset", "cluster_option", "L"])
            
            if len(existing_entries) < 1:
                print(f"no data yet on {dataset} with {cluster_option} at Lipshitz={L}")
                complete = False
                continue

            print(f"\n    Lipschitz = {L}")
            keys = ["convergence_iter", "optimum", "time"]
            for key in keys:
                data = [float(entry[key]) for entry in existing_entries]
                mean = np.mean(data)
                std = np.std(data)

                print(f"    {key}{' '*(20-len(key))} M = {mean:.2f}     SD = {std:.2f}")

            energy_lists_by_run = []
            for entry in existing_entries:
                with open(os.path.join(args.output_path, entry["energy_list_file"]), "r") as f:
                    energy_lists_by_run.append(json.loads(f.read()))

            max_len = max([int(entry["convergence_iter"]) for entry in existing_entries])
            energy_array_by_run = np.zeros((len(existing_entries), max_len))
            for i, energy_list in enumerate(energy_lists_by_run):
                last_value = energy_list[-1]
                for iter in range(len(energy_list), max_len):
                    energy_list.append(last_value)
                energy_array_by_run[i] = energy_list
                
            energy_list_by_L[L] = {
                # "mean": np.mean(energy_array_by_run, axis=1),
                "mean": energy_array_by_run[0],
                "std": np.std(energy_array_by_run, axis=1),
            }

        if complete:
            save_dir = os.path.join(args.output_path, dataset)
            save_path = os.path.join(save_dir, f"{cluster_option}_"+"Lipschitz_plot{suffix}.png")
            plot_Lipschitz_convergence(save_path, energy_list_by_L)
            plot_Lipschitz_convergence(save_path, energy_list_by_L, yscale_log=True)